In [ ]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Sample

In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Contoh dataset (train_df dan test_df)
train_df = pd.DataFrame({
    'LotArea': [9600, None, 12000, 8500],
    'SalePrice': [200000, 150000, None, 130000]
})

test_df = pd.DataFrame({
    'LotArea': [None, 8500, 6000, None],
    'SalePrice': [None, 170000, 120000, None]
})

# Menggabungkan train_df dan test_df untuk transformasi bersama
combined_df = pd.concat([train_df, test_df], keys = ['train', 'test'])

# Inisialisasi SimpleImputer (misalnya menggunakan mean untuk nilai numerik)
imputer = SimpleImputer(strategy='mean')

# Terapkan imputasi pada kolom numerik
combined_df[['LotArea', 'SalePrice']] = imputer.fit_transform(combined_df[['LotArea', 'SalePrice']])
print("Combine DataFrame:")
print(combined_df)

# Memisahkan kembali train_df dan test_df setelah transformasi
train_df = combined_df.xs(key = 'train')
test_df = combined_df.xs(key = 'test')

# Hasil imputasi
print("\nTrain DataFrame:")
print(train_df)
print("\nTest DataFrame:")
print(test_df)


# Sample2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Dataset contoh
data = {
    'size': [1500, 1800, 2400, 3000, 3500, None],
    'location': ['City', 'Suburb', 'City', 'Suburb', 'City', 'Suburb'],
    'price': [400000, 450000, 500000, 550000, 600000, 650000]
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Memisahkan fitur dan target
X = df.drop('price', axis=1)
y = df['price']

# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Mendefinisikan preprocessing untuk fitur numerik dan kategorikal
numeric_features = ['size']
categorical_features = ['location']

# Membuat transformer untuk preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputasi untuk missing values
    ('scaler', StandardScaler())  # Normalisasi data numerik
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputasi untuk missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
])

# Membuat ColumnTransformer untuk menerapkan preprocessing pada kolom yang tepat
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Membuat pipeline yang menggabungkan preprocessing dengan model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())  # Model regresi Linear
])

# Melatih model dengan data training
model_pipeline.fit(X_train, y_train)

# Prediksi harga rumah pada data testing
y_pred = model_pipeline.predict(X_test)

# Menghitung error (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


# Sample8